In [1]:
import pandas as pd
from pipeline import Pipeline, build_csv
from stop_words import stop_words
import string
import json
import io
import datetime
import csv

In [2]:
pipeline = Pipeline()

In [3]:
@pipeline.task()
def file_to_json():
    with open('hn_stories_2014.json', "r") as f:
        stories = json.load(f)
      #  print(stories)- to see the file to know to pull out idx (eg.stories)
        stories = stories['stories']
    return stories

In [4]:
@pipeline.task(depends_on = file_to_json)
def filter_stories(stories):
    filter_stories = []
    for each in stories:
        if each['points']>50 and each['num_comments']>1 and not each['title'].startswith('Ask HN'):
            filter_stories.append(each)
        else:
            pass
    return filter_stories       

In [5]:
#len(story)

In [6]:
@pipeline.task(depends_on = filter_stories)
def json_to_csv(stories):
    lines = []
    for each in stories:
#         story = each['objectID'],  datetime.datetime.strptime(each['created_at'], '%Y-%m-%dT%H:%M:%SZ'), each['url'], each['points'], each['title']
#         lines.append(story)
#         set it as a tuple (double bracket) and put it in the append - that way you don't have a variable - more robust!!
        lines.append((each['objectID'],  datetime.datetime.strptime(each['created_at'], '%Y-%m-%dT%H:%M:%SZ'), each['url'], each['points'], each['title']))
    return build_csv(lines, ['objectID', 'created_at', 'url', 'points', 'title'], io.StringIO())

In [7]:
#this is a image of the generator not still working the generator
#print(*csv, sep='\n')

In [8]:
@pipeline.task(depends_on = json_to_csv)

def extract_titles(csv_file):
    reader = csv.reader(csv_file)
    header = next(reader)
    idx = header.index('title')
    return (row[idx] for row in reader)    

#this is incorrect cause you are not using your generator on the for loop, need the for loop in the generatr
#     uniques = []
#     for line in reader:
#         uniques.append(line['title'])
#     yield (uniques)
#title = extract_titles(csv)

In [9]:
@pipeline.task(depends_on = extract_titles)
def clean_titles(idx):
    for each in idx:
        yield each.lower().translate(str.maketrans('', '', string.punctuation + '–,“,—,”,’,‘,…,')).st

# replace takes up a lot more memory
#        yield each.lower().translate(str.maketrans('', '', string.punctuation)).replace('‘', '').replace('’', '').replace('–', ' ').replace('”', '').replace('“', '').replace('…', '').strip()

        
#        print(each.lower().translate(str.maketrans('', '', string.punctuation)).replace('‘', '').replace('’', '').replace('–', ' ').replace('”', '').replace('“', '').strip() )
#        print (each.lower().replace('[{}]'.format(string.punctuation), '').replace('‘', '').replace('’', ''))

#     using a yield hold on to each list and runs in again and again like appending it to a list but without bringing in the data
#     list=[]
#     for each in idx:
#         list.append(each.lower().translate(str.maketrans('', '', string.punctuation)).replace('‘', '').replace('’', '').replace('–', ' ').replace('”', '').replace('“', '').replace('…', '').strip())

#     return list

In [10]:
@pipeline.task(depends_on = clean_titles)
def build_keyword_dictionary(idx):
    counts = {}
    for row in idx:
         for word in row.split():
            if word not in stop_words:
                if word not in counts:
                    counts[word] = 0
                counts[word] += 1
    return sorted(counts.items(), key=lambda x: (-x[1], x[0]))


In [11]:
@pipeline.task(depends_on = build_keyword_dictionary)
# def top_100(dictionary):
#     dict1 = dictionary[:99]
#     tup = tuple(dict1)
#     return tup
def top_100(dictionary):
    return tuple(dictionary[:99])



In [12]:
ran = pipeline.run()
print(ran[top_100])

(('new', 185), ('google', 167), ('bitcoin', 101), ('open', 93), ('programming', 90), ('web', 89), ('data', 86), ('video', 79), ('python', 76), ('code', 72), ('facebook', 72), ('released', 71), ('using', 71), ('2013', 65), ('javascript', 65), ('free', 64), ('game', 64), ('source', 64), ('internet', 63), ('c', 59), ('linux', 59), ('microsoft', 59), ('app', 58), ('dont', 57), ('pdf', 55), ('language', 54), ('work', 54), ('2014', 52), ('software', 52), ('startup', 51), ('apple', 50), ('make', 50), ('use', 50), ('security', 48), ('time', 48), ('yc', 48), ('github', 45), ('nsa', 45), ('windows', 44), ('1', 41), ('heartbleed', 41), ('like', 41), ('way', 41), ('world', 41), ('computer', 40), ('project', 40), ('design', 37), ('git', 37), ('ios', 37), ('twitter', 37), ('users', 37), ('big', 36), ('ceo', 36), ('developer', 36), ('life', 36), ('os', 36), ('vs', 36), ('day', 35), ('android', 34), ('online', 34), ('simple', 34), ('court', 33), ('years', 33), ('api', 32), ('apps', 32), ('browser', 32